In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import tensorflow as tf

import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, ZeroPadding2D


In [3]:
train_data = pd.read_csv('fashion-mnist_train.csv')
test_data = pd.read_csv('fashion-mnist_test.csv')

train_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train_data = np.array(train_data, dtype= 'float32')
test_data = np.array(test_data, dtype= 'float32')

In [5]:
#Slicing the labels and Normalizing the pixel data.
X_train_orig = train_data[:, 1:]/ 255
y_train_orig = train_data[:, 0]

X_test = test_data[:, 1:]/ 255
Y_test = test_data[:, 0]

In [6]:

X_train, X_validate, y_train, y_validate = train_test_split(X_train_orig, y_train_orig, test_size = 0.2, random_state=42)

In [7]:
#Data Preprocessing-reshape, labelling

image_rows = 28
image_cols = 28
input_shape = (image_rows, image_cols, 1)

X_train = X_train.reshape(X_train.shape[0], image_rows, image_cols, 1)
X_test = X_test.reshape(X_test.shape[0], image_rows, image_cols, 1)
X_validate = X_validate.reshape(X_validate.shape[0], image_rows, image_cols, 1)

print('x_train shape: {}'.format(X_train.shape))
print('x_test shape: {}'.format(X_test.shape))
print('x_validate shape: {}'.format(X_validate.shape))

# One-hot encoding
y_train = to_categorical(y_train, 10)
y_validate = to_categorical(y_validate, 10)
y_test = to_categorical(Y_test, 10)

x_train shape: (48000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)
x_validate shape: (12000, 28, 28, 1)


In [8]:
cnn_final = Sequential()

cnn_final.add(Conv2D(32, 3, padding='same', activation='relu', input_shape=input_shape))
cnn_final.add(Dropout(0.5))
BatchNormalization()

cnn_final.add(Conv2D(64, 3, padding='same', activation='relu'))
cnn_final.add(MaxPooling2D(2))
BatchNormalization()

cnn_final.add(Conv2D(128, 3, padding='same', activation ='relu'))
cnn_final.add(Dropout(0.5))
BatchNormalization()
cnn_final.add(Conv2D(256, 3, padding='same', activation ='relu'))
cnn_final.add(MaxPooling2D(2, 2))
BatchNormalization()

cnn_final.add(Conv2D(512, 3, padding='same', activation ='relu'))
cnn_final.add(Dropout(0.5))
cnn_final.add(MaxPooling2D(2, 2))
BatchNormalization()

cnn_final.add(Dropout(0.5))

cnn_final.add(Flatten())

cnn_final.add(Dense(256, activation='relu'))
cnn_final.add(Dropout(0.5))
BatchNormalization()
cnn_final.add(Dense(512, activation='relu'))
cnn_final.add(Dropout(0.5))
BatchNormalization()
cnn_final.add(Dense(1024, activation='relu'))
cnn_final.add(Dropout(0.5))
BatchNormalization()
cnn_final.add(Dense(10, activation='softmax'))

In [9]:
cnn_final.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 dropout (Dropout)           (None, 28, 28, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 dropout_1 (Dropout)         (None, 14, 14, 128)       0         
                                                        

In [10]:

cnn_final.compile(optimizer=Adam(learning_rate = 0.0001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history_final = cnn_final.fit(X_train, y_train,
                    validation_data=(X_validate, y_validate), epochs=50, batch_size=300,
                    validation_steps=12000//150,
                    steps_per_epoch = 48000//400)



Epoch 1/50
120/120 [==============================] - 1429s 12s/step - loss: 1.9627 - accuracy: 0.2364 - val_loss: 1.6673 - val_accuracy: 0.5258
Epoch 2/50
120/120 [==============================] - 1352s 11s/step - loss: 1.2120 - accuracy: 0.5057
Epoch 3/50
120/120 [==============================] - 1380s 12s/step - loss: 0.9079 - accuracy: 0.6543
Epoch 4/50
120/120 [==============================] - 1394s 12s/step - loss: 0.7636 - accuracy: 0.7116
Epoch 5/50
120/120 [==============================] - 1354s 11s/step - loss: 0.6988 - accuracy: 0.7337
Epoch 6/50
120/120 [==============================] - 1386s 12s/step - loss: 0.6586 - accuracy: 0.7474
Epoch 7/50
120/120 [==============================] - 1420s 12s/step - loss: 0.6303 - accuracy: 0.7600
Epoch 8/50
120/120 [==============================] - 1079s 9s/step - loss: 0.5842 - accuracy: 0.7747
Epoch 9/50
120/120 [==============================] - 738s 6s/step - loss: 0.5628 - accuracy: 0.7841
Epoch 10/50
 51/120 [===========>.

In [ ]:
scoreFinal = cnn_final.evaluate(X_test, y_test, verbose=0)
print('Test loss:', scoreFinal[0])
print('Test accuracy:', scoreFinal[1]*100)



In [ ]:
y_pred = cnn_final.predict_classes(X_test, verbose=0)
score = cnn_final.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1] * 100)

In [ ]:
cnn_final.save("cnn_finalModel.h5")